<a href="https://colab.research.google.com/github/sachinkun21/Fuel_Efficiency_Forecast/blob/master/L%26T_Assignment_with_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Collect Data → Explore and Transform Data (EDA) → Develop Model → Evaluate Model → Deploy Model → Maintain

# INTRODUCTION



* **EDA (Exploratory Data Analysis):**We will start with **data description and cleaning**, then we will visualize our data to understand better.

* After that, we will focus on **time series prediction** to predict fuel level. 
* For time series prediction, we will use **ARIMA** method. 
 
 <br> <font color='blue'> Content: 
    * [Load the Data](#1)
    * [Data Description](#2)
    * [Data Cleaning](#3)
    * [Data Visualization](#4)
    * [Time Series Prediction with ARIMA](#5)
        * [ What is Time Series ?](#6)
        * [Stationarity of a Time Series](#7)
        * [Make a Time Series Stationary](#8)
            * Moving Average method
            * Differencing method
        * [Forecasting a Time Series](#9)
    * [Conclusion](#10)


In [0]:
# Importing Libraries and Modules

import numpy as np # for calculations
import pandas as pd # data processing, Reading file I/O (e.g. pd.read_excel)
import seaborn as sns # visualization library
import matplotlib.pyplot as plt # visualization library
import plotly.plotly as py # visualization library
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
init_notebook_mode(connected=True) 
import plotly.graph_objs as go # plotly graphical object

# Data set is available in Google Drive ;/content/drive/My Drive/Assignment L&T' folder.

import os

import warnings            
warnings.filterwarnings("ignore") # if there is a warning after some codes due to deprecation, this will avoid us to see them.
plt.style.use('ggplot') # style set for plots


In [0]:
# Mounting Drive for Importing Dataset
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# printing contents of Directory
print(os.listdir("/content/drive/My Drive/Assignment L&T"))

<a id="1"></a> <br>
## Loading the Data
As the environment is set and Necessary modules have been Imported, we will proceed and read the dataset into pandas DataFrame for Analysis


In [0]:
# Reading data from excel file into Pandas DataFrame
df = pd.read_excel("/content/drive/My Drive/Assignment L&T/data_set.xlsx")

#Displaying the first 5 rows of DataFrame
df.head()


In [0]:
df[(df['Level of Fuel']>0) & (df['ON/OFF']>0)].head(10)

<a id="2"></a> <br>
## Data Description
**DESCRIPTION OF FEATURE VARIABLES:**
* Company ID: Represents Machine Type
* DateTime: Contains the Date and Time when observation was Recorded
* ON/OFF: Represents the Working State of Machine, i.e whether it is switched On or Off.
* PF: 
* Kilowatt: Describes the measure of electrical energy Generated
* KWH: kWH = kW * Hours. also represents Electric Energy Generated.
ErrorCode: Represent the condition in which Machine is working

### EDA:
Descriptive Analysis

In [0]:
#Printing Column Names
df.columns

In [0]:
df.shape

There are 72791 data Points or measurements available with 8 Feature Variables

In [0]:
df.info()

DateTime is of DateTime datatype while rest of the features are Numeric.<br>
Further, there is no Null/Missing Value in any of the Variables.

#### Printing Numerical Description of all the Variables.

In [0]:
df.describe()

### Printing Number of Measurements for each CompanyId i.e Device Generator

In [0]:
df['Company Id'].value_counts()

Printing Error Code Counts for each Unqiue Code in Dataset

In [0]:
df['ON/OFF'].value_counts()

In [0]:
df.ErrorCode.value_counts()

Before we proceed with Cleaning and Analysis, Let's see for how many measurements the Level of Fuel was 0 i.e empty Tank

In [0]:
print(df[df['Level of Fuel']==0].count()[0],'Points i.e', (df[df['Level of Fuel']==0].count()[0]/len(df))*100,' Percent of total measurements')

So, around 44.65% of total measurements reported Empty Tank

<a id="3"></a> <br>
## Data Cleaning:
* If Dataset contained Null/Missing values, Instead of usign them, we should drop or Impute them based on count. It will not only remove the uncertainty but it also help us in visualization.
* Since our dataset does not contain NaN value we will not or impute anything with further analysis. 
    
* According to exploratory data analysis and visualization, we will pick certain variables in order to examine them deeper. 

### Outlier Detection Using BoxPlot

In [0]:
f, ax = plt.subplots(figsize=(10, 8))
fig = sns.boxplot(x='Company Id', y="KiloWatt", data=df)
plt.show()

<a id="4"></a> <br>
## Data Visualization
* Lets start with basics of visualization for understanding data.
    * We will plot pairWise relation among each Variable for Status Type ON and OFF seperately in each subplot
    * Top target countries
    * Top 10 aircraft series
    * Takeoff base locations (Attacjk countries)
    * Target locations (If you do not understand methods of pyplot look at my pyplot tutorial: https://www.kaggle.com/kanncaa1/plotly-tutorial-for-beginners)
    * Bombing paths
    * Theater of Operations
    * Weather station locations

**Let's see how the numerical data is distributed**

In [0]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

df.hist(bins=20, figsize=(14,10), color='#E14906', edgecolor ='black')
plt.show()

In [0]:
sns.set(style="ticks")

sns.pairplot(df, hue="ON/OFF", palette="Set1")
plt.show()

In [0]:
plt.figure(figsize=(12,8))
sns.countplot(df['Company Id'])
plt.xticks(rotation=90)
plt.show()

In [0]:
plt.subplots( figsize=(20,8))

sns.lineplot(x='KiloWatt', y = 'KWH', data = df)

plt.show()

In [0]:
f, ax = plt.subplots(1,2, figsize=(20,8))

colors = [ "#64FE2E","#FA5858"]
labels ="Status On", "Status OFF"

plt.suptitle('Information on Status of Each Machine Type', fontsize=20)

df["ON/OFF"].value_counts().plot.pie(explode=[0,0.05], autopct='%1.2f%%', ax=ax[0], shadow=True, colors=colors, 
                                             labels=labels, fontsize=15, startangle=45)



ax[0].set_xlabel('ON/OFF Status percentage', fontsize=14)
ax[0].set_ylabel('')

palette = ["#64FE2E", "#FA5858"]

sns.barplot(x="Company Id", y="Level of Fuel", hue="ON/OFF", data=df, palette=palette, estimator=lambda x: sum(x)/len(x))
ax[1].set(ylabel="(Mean Value of Level of Fuel)")
ax[1].set_xticklabels(df["Company Id"].unique(), rotation=0, rotation_mode="anchor")
plt.show()

In [0]:
f, ax = plt.subplots(1,2, figsize=(20,8))

colors = ["#FA5858", "#64FE2E"]
labels ="Error Code 0", "Error Code 1"

plt.suptitle('Information on ErrorCode of Each Machine Type', fontsize=20)

df["ErrorCode"].value_counts().plot.pie(explode=[0,0.10], autopct='%1.2f%%', ax=ax[0], shadow=True, colors=colors, 
                                             labels=labels, fontsize=15, startangle=45)



ax[0].set_xlabel('ON/OFF Status percentage', fontsize=14)
ax[0].set_ylabel('')


palette = ["#64FE2E", "#FA5858"]

sns.barplot(x="Company Id", y="Level of Fuel", hue="ErrorCode", data=df, palette=palette, estimator=lambda x:sum(x)/len(x))
ax[1].set(ylabel="(Mean Value of Level of Fuel)")
ax[1].set_xticklabels(df["Company Id"].unique(), rotation=0, rotation_mode="anchor")
plt.show()

In [0]:
f, ax = plt.subplots(1,2, figsize=(20,8))

palette = ["#64FE2E", "#FA5858"]

sns.barplot(x="Company Id", y="KWH", hue="ErrorCode", data=df, palette=palette, estimator=lambda x:sum(x)/len(x), ax=ax[0])
ax[0].set(ylabel="(Mean Value of Power Generated)")
ax[0].set_xticklabels(df["Company Id"].unique(), rotation=0, rotation_mode="anchor")


colors = ["#FA5858", "#64FE2E"]
labels ="Error Code 0", "Error Code 1"

plt.suptitle('Information on ErrorCode of Each Machine Type', fontsize=20)

df["ErrorCode"].value_counts().plot.pie(explode=[0,0.10], autopct='%1.2f%%', ax=ax[1], shadow=True, colors=colors, 
                                             labels=labels, fontsize=15, startangle=25)



ax[1].set_xlabel('ON/OFF Status percentage', fontsize=14)
ax[1].set_ylabel('')


plt.show()

## Plotting the Correlation Matrix of Dataset

In [0]:
plt.figure(figsize=(14,8))
sns.heatmap(df.corr(), annot = True,cmap="OrRd")
plt.show()

In [0]:
StatusOn_df = df.loc[df["ON/OFF"] == 1]

list_of_CompanyID = df["Company Id"].unique().tolist()
print(list_of_CompanyID)

In [0]:
# StatusOn_df = df.loc[df["ON/OFF"] == 1]

# list_of_CompanyID = df["Company Id"].unique().tolist()

# # Get the balances by jobs
# CompanyID_25502921 = StatusOn_df["KWH"].loc[StatusOn_df["Company Id"] == list_of_CompanyID[0]].values
# CompanyID_25750275 = StatusOn_df["KWH"].loc[StatusOn_df["Company Id"] == list_of_CompanyID[1]].values
# CompanyID_25921010 = StatusOn_df["KWH"].loc[StatusOn_df["Company Id"] == list_of_CompanyID[2]].values
# CompanyID_25921355 = StatusOn_df["KWH"].loc[StatusOn_df["Company Id"] == list_of_CompanyID[3]].values
# CompanyID_25927159 = StatusOn_df["KWH"].loc[StatusOn_df["Company Id"] == list_of_CompanyID[4]].values




# companies = [CompanyID_25502921,CompanyID_25750275,CompanyID_25921010,CompanyID_25921355,CompanyID_25927159]
# list_of_CompanyID = [str(id) for id in companies]



# colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)',
#           'rgba(44, 160, 101, 0.5)', 'rgba(255, 65, 54, 0.5)', 
#           'rgba(174, 229, 56, 0.5)']

# traces = []

# for xd, yd, cls in zip(list_of_CompanyID, companies, colors):
#         traces.append(go.Box(
#             y=yd,
#             name=xd,
#             boxpoints='all',
#             jitter=0.5,
#             whiskerwidth=0.2,
#             fillcolor=cls,
#             marker=dict(
#                 size=2,
#             ),
#             line=dict(width=1),
#         ))

# layout = go.Layout(
#     title='Distribution of Power Generated per CompanyID',
#     yaxis=dict(
#         autorange=True,
#         showgrid=True,
#         zeroline=True,
#         dtick=5,
#         gridcolor='rgb(255, 255, 255)',
#         gridwidth=1,
#         zerolinecolor='rgb(255, 255, 255)',
#         zerolinewidth=2,
#     ),
#     margin=dict(
#         l=40,
#         r=30,
#         b=80,
#         t=100,
#     ),
#     paper_bgcolor='rgb(224,255,246)',
#     plot_bgcolor='rgb(251,251,251)',
#     showlegend=False
# )

# fig = go.Figure(data=traces, layout=layout)
# iplot(fig)


In [0]:
# # Aircraft Series
# data = df['Company Id'].value_counts()

# data = [go.Bar(
#             x=data.index,
#             y=data.values,
#             hoverinfo = 'text',
#             marker = dict(color = 'rgba(177, 14, 22, 0.5)',
#                              line=dict(color='rgb(0,0,0)',width=1.5)),
#     )]

# layout = dict(
#     title = 'Company Id',
# )
# fig = go.Figure(data=data, layout=layout)
# iplot(fig)

## Feature Engineering
    

### In this section, we will Calculate Fuel consumed using following steps:

1.   We will seperate Data for each machine in a new DataFrame.
2.   For Each Dataframe formed, We will calculate Fuel Consumption for Each Record/Datapoint by subtracting 'Fuel Level' in the record/Datapoint just before that Datapoint.<br>
For Detailed Explanation see the `fuel_consumption` Function and It's documentaition given below.
3. Next We Will Resample the Datapoints in each Dataframe using 'Day' as Time Period(i.e aggregating the per 15 seconds caluculation into Daily Calculation) and sum the Fuel Consumption over each day.
Thus we will get Daily Fuel Consumption.




In [0]:
df.head()

Creating One DataFrame for Each Company ID i.e Machine Code

In [0]:
list_of_CompanyID = df["Company Id"].unique().tolist()
print(list_of_CompanyID)

In [0]:
df_machine1=df.loc[df['Company Id']==list_of_CompanyID[0]].reset_index()
df_machine2=df.loc[df['Company Id']==list_of_CompanyID[1]].reset_index()
df_machine3=df.loc[df['Company Id']==list_of_CompanyID[2]].reset_index()
df_machine4=df.loc[df['Company Id']==list_of_CompanyID[3]].reset_index()
df_machine5=df.loc[df['Company Id']==list_of_CompanyID[4]].reset_index()
df_machine3.head()

In [0]:
df_machine3.loc[df_machine3['Level of Fuel']>0].head(5)

In [0]:
len(df_machine1)

## Function for calulation Fuel consumption at Each DataPoint/record.

In [0]:
# df_machine3.at[107, 'Fuel Consumption'] = df_machine3.iloc[106]['Level of Fuel']-df_machine3.iloc[107]['Level of Fuel']
# df_machine3.iloc[107]['Fuel Consumption']

In [0]:
def fuel_consumption(dataframe):
  for i in range(1,len(dataframe)):
      dataframe.at[i, 'Fuel Consumption']=dataframe.iloc[i-1]['Level of Fuel']-dataframe.iloc[i]['Level of Fuel']
    
      

In [0]:
fuel_consumption(df_machine1)
fuel_consumption(df_machine2)
fuel_consumption(df_machine3)
fuel_consumption(df_machine4)
fuel_consumption(df_machine5)


In [0]:
machine3_positive_consumption = df_machine3.loc[df_machine3['Fuel Consumption']>0]
print('Total Instances where fuel was consumed:',len(machine3_positive_consumption))
print('\n')
print('Top 5 rows of such instances:\n')
machine3_positive_consumption.head()

In [0]:
machine3_positive_consumption['Fuel Consumption'].quantile([.1, .25, .5, .75, 0.85,0.87,0.90,0.95, 0.99])

In [0]:
print('Count: ',len(df_machine3.loc[(df_machine3['Fuel Consumption']<0)]))
df_machine3.loc[(df_machine3['Fuel Consumption']<0)].head()

In [0]:
df_machine3.loc[(df_machine3['Fuel Consumption']<0) & (df_machine3['ErrorCode']!=0)].head()

### Linear Regression Model for Machine 3:

In [0]:
X= df_machine3.drop(['DateTime', 'Company Id', 'Fuel Consumption'], axis  = 1)
y=df_machine3['Fuel Consumption']

In [0]:

from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,train_size=0.7,test_size=0.3, random_state=42 )
print(len(X_train), len(X_test))

In [0]:
print(X_train.head())
print(y_train.head())

In [0]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler


from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [0]:

model = LinearRegression()

# lasso_model_cv = GridSearchCV(estimator = lasso_cv, param_grid=params, scoring = 'neg_mean_squared_log_error', 
#                         cv = 5,return_train_score=True,verbose = 1)

In [0]:
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
y_pred=model.predict(X_test)
y_train_pred=model.predict(X_train)


In [0]:
y_pred

In [0]:
from sklearn.metrics import r2_score
print('test_score %2.2f' % r2_score(y_test,y_pred))
print('Train_score %2.2f'% r2_score(y_train,y_train_pred))

In [0]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

<a id="5"></a> <br>
## Time Series Prediction with ARIMA
* We will use most used method ARIMA
* ARIMA :  AutoRegressive Integrated Moving Average. I will explain it detailed  at next parts.
* The way that we will follow: 
    * What is Time Series ?
    * Stationarity of a Time Series
    * Make a Time Series Stationary?
    * Forecasting a Time Series

<a id="6"></a> <br>
### What is time series?
* Time series is a collection of data points that are collected at constant time intervals.
* It is time dependent.
* Most of time series have some form of **seasonality trends**. For example, if we sale ice cream, most probably there will be higher sales in summer seasons. Therefore, this time series has seasonality trends.
* Another example, lets think we dice one time every day during 1 year. As you guess, there will be no scenario like that number six is appeared mostly in summer season or number five is mostly appeared in January. Therefore, this time series does not have seasonality trends.


<a id="7"></a> <br>
### Stationarity of a Time Series
* There are three basic criterion for a time series to understand whether it is stationary series or not.
    * Statistical properties of time series such as mean, variance should remain constant over time to call **time series is stationary**
        * constant mean
        * constant variance
        * autocovariance that does not depend on time. autocovariance is covariance between time series and lagged time series.
* Lets visualize and check seasonality trend of our time series.

In [0]:
# Mean temperature of Bindikuri area
plt.figure(figsize=(22,10))
plt.plot(df.DateTime,df.KiloWatt)
plt.title("Mean Temperature of Bindukuri Area")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.show()




In [0]:
# lets create time series from weather 
timeSeries = weather_bin.loc[:, ["Date","MeanTemp"]]
timeSeries.index = timeSeries.Date
ts = timeSeries.drop("Date",axis=1)

* As you can see from plot above, our time series has seasonal variation. In summer, mean temperature is higher and in winter mean temperature is lower for each year.
* Now lets check stationary of time series. We can check stationarity using the following methods: 
    * Plotting Rolling Statistics: We have a window lets say window size is 6 and then we find rolling mean and variance to check stationary.
    * Dickey-Fuller Test: The test results comprise of a **Test Statistic** and some **Critical Values** for difference confidence levels. If the **test statistic** is less than the **critical value**, we can say that time series is stationary.

In [0]:
# adfuller library 
from statsmodels.tsa.stattools import adfuller
# check_adfuller
def check_adfuller(ts):
    # Dickey-Fuller test
    result = adfuller(ts, autolag='AIC')
    print('Test statistic: ' , result[0])
    print('p-value: '  ,result[1])
    print('Critical Values:' ,result[4])
# check_mean_std
def check_mean_std(ts):
    #Rolling statistics
    rolmean = pd.rolling_mean(ts, window=6)
    rolstd = pd.rolling_std(ts, window=6)
    plt.figure(figsize=(22,10))   
    orig = plt.plot(ts, color='red',label='Original')
    mean = plt.plot(rolmean, color='black', label='Rolling Mean')
    std = plt.plot(rolstd, color='green', label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Standard Deviation')
    plt.legend()
    plt.show()
    
# check stationary: mean, variance(std)and adfuller test
check_mean_std(ts)
check_adfuller(ts.MeanTemp)


* Our first criteria for stationary is constant mean. So we fail because mean is not constant as you can see from plot(black line) above . (no stationary)
* Second one is constant variance. It looks like constant. (yes stationary)
* Third one is that If the **test statistic** is less than the **critical value**, we can say that time series is stationary. Lets look:
    * test statistic = -1.4 and critical values = {'1%': -3.439229783394421, '5%': -2.86545894814762, '10%': -2.5688568756191392}. Test statistic is bigger than the critical values. (no stationary)
* As a result, we sure that our time series is not stationary.
* Lets make time series stationary at the next part.

<a id="8"></a> <br>
### Make a Time Series Stationary?
* As we mentioned before, there are 2  reasons behind non-stationarity of time series
    * Trend: varying mean over time. We need constant mean for stationary of time series.
    * Seasonality: variations at specific time. We need constant variations for stationary of time series.
* First solve **trend(constant mean)** problem
    * Most popular method is moving average.
        * Moving average: We have window that take the average over the past 'n' sample. 'n' is window size.

In [0]:
# Moving average method
window_size = 6
moving_avg = pd.rolling_mean(ts,window_size)
plt.figure(figsize=(22,10))
plt.plot(ts, color = "red",label = "Original")
plt.plot(moving_avg, color='black', label = "moving_avg_mean")
plt.title("Mean Temperature of Bindukuri Area")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.show()

In [0]:
ts_moving_avg_diff = ts - moving_avg
ts_moving_avg_diff.dropna(inplace=True) # first 6 is nan value due to window size

# check stationary: mean, variance(std)and adfuller test
check_mean_std(ts_moving_avg_diff)
check_adfuller(ts_moving_avg_diff.MeanTemp)

* Constant mean criteria: mean looks like constant as you can see from plot(black line) above . (yes stationary)
* Second one is constant variance. It looks like constant. (yes stationary)
* The test statistic is smaller than the 1% critical values so we can say with 99% confidence that this is a stationary series. (yes stationary)
* We achieve stationary time series. However lets look at one more method to avoid trend and seasonality.
    * Differencing method: It is one of the most common method. Idea is that take difference between time series and shifted time series. 


In [0]:
# differencing method
ts_diff = ts - ts.shift()
plt.figure(figsize=(22,10))
plt.plot(ts_diff)
plt.title("Differencing method") 
plt.xlabel("Date")
plt.ylabel("Differencing Mean Temperature")
plt.show()

In [0]:
ts_diff.dropna(inplace=True) # due to shifting there is nan values
# check stationary: mean, variance(std)and adfuller test
check_mean_std(ts_diff)
check_adfuller(ts_diff.MeanTemp)

* Constant mean criteria: mean looks like constant as you can see from plot(black line) above . (yes stationary)
* Second one is constant variance. It looks like constant. (yes stationary)
* The test statistic is smaller than the 1% critical values so we can say with 99% confidence that this is a stationary series. (yes stationary)

<a id="9"></a> <br>
### Forecasting a Time Series
* We learn two different methodsthat are **moving average and differencing** methods to avoid trend and seasonality problem
* For prediction(forecasting) we will use ts_diff time series that is result of differencing method. There is no reason I only choose it.
* Also prediction method is ARIMA that is Auto-Regressive Integrated Moving Averages.
    * AR: Auto-Regressive (p): AR terms are just lags of dependent variable. For example lets say p is 3, we will use  x(t-1), x(t-2) and x(t-3) to predict x(t)
    * I: Integrated (d): These are the number of nonseasonal differences. For example, in our case we take the first order difference. So we pass that variable and put d=0 
    * MA: Moving Averages (q): MA terms are lagged forecast errors in prediction equation.
* (p,d,q) is parameters of ARIMA model.
* In order to choose p,d,q parameters we will use two different plots.
    * Autocorrelation Function (ACF): Measurement of the correlation between time series and lagged version of time series. 
    * Partial Autocorrelation Function (PACF): This measures the correlation between the time series and lagged version of time series but after eliminating the variations already explained by the intervening comparisons. 

In [0]:
# ACF and PACF 
from statsmodels.tsa.stattools import acf, pacf
lag_acf = acf(ts_diff, nlags=20)
lag_pacf = pacf(ts_diff, nlags=20, method='ols')
# ACF
plt.figure(figsize=(22,10))

plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

# PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

* Two dotted lines are the confidence interevals. We use these lines to determine the ‘p’ and ‘q’ values
    * Choosing p: The lag value where the PACF chart crosses the upper confidence interval for the first time. p=1.
    * Choosing q: The lag value where the ACF chart crosses the upper confidence interval for the first time. q=1.
* Now lets use (1,0,1) as parameters of ARIMA models and predict  
    * ARIMA: from statsmodels libarary
    * datetime: we will use it start and end indexes of predict method

In [0]:
# ARIMA LİBRARY
from statsmodels.tsa.arima_model import ARIMA
from pandas import datetime

# fit model
model = ARIMA(ts, order=(1,0,1)) # (ARMA) = (1,0,1)
model_fit = model.fit(disp=0)

# predict
start_index = datetime(1944, 6, 25)
end_index = datetime(1945, 5, 31)
forecast = model_fit.predict(start=start_index, end=end_index)

# visualization
plt.figure(figsize=(22,10))
plt.plot(weather_bin.Date,weather_bin.MeanTemp,label = "original")
plt.plot(forecast,label = "predicted")
plt.title("Time Series Forecast")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.show()

* lets predict and visualize all path and find mean squared error

In [0]:
# predict all path
from sklearn.metrics import mean_squared_error
# fit model
model2 = ARIMA(ts, order=(1,0,1)) # (ARMA) = (1,0,1)
model_fit2 = model2.fit(disp=0)
forecast2 = model_fit2.predict()
error = mean_squared_error(ts, forecast2)
print("error: " ,error)
# visualization
plt.figure(figsize=(22,10))
plt.plot(weather_bin.Date,weather_bin.MeanTemp,label = "original")
plt.plot(forecast2,label = "predicted")
plt.title("Time Series Forecast")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.savefig('graph.png')

plt.show()

<a id="10"></a> <br>
# Conclusion
* In this tutorial, I want ot make tutorial about ARIMA and make some visualization before it.
* We learn how to make map plots with pyplot. 
* We learn how to make time series forecast.
* **If you have any question advise or feedback, I will be very happy to hear it**